In [1]:
from keras.models import Model
from keras.layers import Input,Dense,LSTM,GRU,Embedding,Bidirectional,RepeatVector,Concatenate,Activation,Dot,Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend as k
import pandas as pd

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/data.csv')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /data.csv


In [0]:
data=pd.read_csv('/content/data.csv')

In [0]:
data.columns=['utterance','emotion']

In [0]:
data['emotion']=data.emotion.replace({1:'angry',2:'sad',3:'happy',4:'worry',0:'neutral'})
data_worry=data[data.emotion=='worry']
data_angry=data[data.emotion=='angry'][:1000]
data_sad=data[data.emotion=='sad'][:1000]
data_happy=data[data.emotion=='happy'][:1000]
data_neutral=data[data.emotion=='neutral'][:1000
                                           ]
final_data=pd.concat([data_worry,data_neutral,data_sad,data_happy,data_angry])
from sklearn.utils import shuffle
final_data=shuffle(final_data)
final_data.to_csv('data.csv')

In [39]:
final_data.emotion.value_counts()

worry      1108
neutral    1000
sad        1000
happy      1000
angry      1000
Name: emotion, dtype: int64

In [0]:
x=final_data.utterance.astype(str)
y=final_data.emotion.astype(str)

In [0]:

out=[]
labels=[]

for w in y:
    
    out.append(w)
for l in out:
    target_text='<START> '+ l +' <END>'
   
    labels.append(target_text)

In [0]:
texts=[]
for line in x:
    texts.append(line)

In [0]:

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer(num_words=40000)

In [0]:
tokenizer.fit_on_texts(texts+labels)

In [45]:
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 9602


In [13]:
!wget http://nlp.stanford.edu/data/glove.6B.zip


--2020-02-01 04:47:45--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-02-01 04:47:45--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-02-01 04:47:45--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [14]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [46]:
print('Indexing word vectors.')
import numpy as np

embeddings_index = {}
f = open('glove.6B.200d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [0]:
num_words_input=len(tokenizer.word_index)+1
word2idx_input=tokenizer.word_index
MAX_NUM_WORDS=40000
num_words=num_words_input
embedding_matrix=np.zeros((num_words,200))
for word,i in word2idx_input.items():
    if i<MAX_NUM_WORDS:
        embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

In [132]:
from keras import preprocessing,utils
import numpy as np
tokenized_questions = tokenizer.texts_to_sequences( texts )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( labels)
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )

padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( labels)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_target_one_hot=np.zeros((len(texts),maxlen_answers,VOCAB_SIZE),dtype='float32')
for i,d in enumerate(padded_answers):
    for t,word in enumerate(d):
        decoder_target_one_hot[i,t,word]=1

# Saving all the arrays to storage
np.save( 'enc_in_data.npy' , encoder_input_data )
np.save( 'dec_in_data.npy' , decoder_input_data )


(5108, 36) 36
(5108, 3) 3


In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
#make sure that we do softmax over time axis = 1
#expected shape N X T X D
# N=no. of samples
#T=sequence length
#D vector dimensionality
def softmax_over_time(x):
    assert(k.ndim(x)>2)
    
    e=k.exp(x-k.max(x,axis=1,keepdims=True))
    s=k.sum(e,axis=1,keepdims=True)
    return e/s  

In [0]:
BATCH_SIZE=64
EPOCHS=1
LATENT_DIM=256
LATENT_DIM_DECODER=256
NUM_SAMPLES=10000
MAX_SEQUENCE_LENGTH=100
MAX_NUM_WORDS=20000
EMBEDDING_DIM=100

In [0]:
#creating embedding layer
embedding_layer=Embedding(VOCAB_SIZE,256,weights=[embedding_matrix],input_length=maxlen_questions)

In [0]:
 ###building model
#setup encoder simple
encoder_input_placeholder=Input(shape=(maxlen_questions,))
x=embedding_layer(encoder_input_placeholder)
encoder=Bidirectional(LSTM(LATENT_DIM,return_sequences=True,dropout=0.3))
encoder_outputs=encoder(x)

In [0]:
decoder_inputs_placeholder=Input(shape=(maxlen_answers,))


In [0]:
decoder_embedding=Embedding(VOCAB_SIZE,256)
decoder_inputs_x=decoder_embedding(decoder_inputs_placeholder)

In [0]:
#attention layer
attn_repeat_layer=RepeatVector(maxlen_questions)
attn_concat_layer=Concatenate(axis=-1)
attn_dense1=Dense(10,activation='tanh')
attn_dense2=Dense(1,activation=softmax_over_time)
attn_dot=Dot(axes=1)#to perform the weighted sum of alpha(t)*h(t)

In [0]:
def one_step_attention(h,st_1):
    #h=h(1)......h(Tx),shape=(Tx,LATEND_DIM*2)
    #ST_1=s(t-1),shape=(LATENT_DIM_DECODER,)
    
    #copy s(t-1) tx times
    #now shape=(Tx,LATENT_DIM_DECODER)
    st_1=attn_repeat_layer(st_1)
    
    #concat all h(t)'s with s(t-1)
    #now shape (tx,LATENT_DIM_DECODER+LATENT_DIM*2)
    x=attn_concat_layer([h,st_1])
    
    #neural net first layer
    x=attn_dense1(x)
    
    #neural net second layer with special softmax over time
    alphas=attn_dense2(x)
    
    #Dot the alphas and h's
    #a.dot(b)=sum over a[t]*b[t]
    
    context = attn_dot([alphas,h])
    return context

In [0]:
#define rest of the decoder(after attention)
from keras import regularizers
decoder_lstm=LSTM(256,return_state=True,dropout=0.25)
decoder_dense=Dense(VOCAB_SIZE,activation='softmax',activity_regularizer=regularizers.l2(0.00001),kernel_regularizer=regularizers.l2(0.00001))

In [0]:
initial_s=Input(shape=(256,),name='s0')
initial_c=Input(shape=(256,),name='c0')
context_last_word_concat_layer=Concatenate(axis=2)

In [0]:
# s and c will reassign after each step
s=initial_s
c=initial_c
outputs=[]

#collect output in a list at first
for t in range(maxlen_answers): #ty times
    #get the context using attention mech
    context=one_step_attention(encoder_outputs,s)
    
    #we need a different layer for each time step
    selector=Lambda(lambda x: x[:,t:t+1])
    xt=selector(decoder_inputs_x)
    
    #combine
    decoder_lstm_input=context_last_word_concat_layer([context,xt])
    
    #pass the combined [context,last word] into lstm
    #along with [s,c]
    #get the new[s,c] and output
    o,s,c=decoder_lstm(decoder_lstm_input,initial_state=[s,c])
    
    #final dense layer to get next word prediction
    decoder_outputs=decoder_dense(o)
    outputs.append(decoder_outputs)

In [0]:
def stack_and_transpose(x):
    # x is a lisy of length T, each element is a batch_size x output_vocab_size tensor
    x=k.stack(x)
    x=k.permute_dimensions(x,pattern=(1,0,2)) # is now batch_size x T x output_vocab_size
    return x

In [0]:
stacker=Lambda(stack_and_transpose)
outputs=stacker(outputs)

In [0]:
model=Model(inputs=[encoder_input_placeholder,decoder_inputs_placeholder,initial_s,initial_c],outputs=outputs)

In [0]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [150]:
#train the model
z=np.zeros((5108,256)) #initial s c
r=model.fit([encoder_input_data,decoder_input_data,z,z],decoder_target_one_hot,batch_size=100,epochs=20,validation_split=0.2)


Train on 4086 samples, validate on 1022 samples
Epoch 1/20
4086/4086 [==============================] - 18s 4ms/step - loss: 2.2476 - acc: 0.5335 - val_loss: 1.0829 - val_acc: 0.6667
Epoch 2/20
4086/4086 [==============================] - 7s 2ms/step - loss: 0.8535 - acc: 0.7184 - val_loss: 0.6544 - val_acc: 0.7316
Epoch 3/20
4086/4086 [==============================] - 7s 2ms/step - loss: 0.6017 - acc: 0.7340 - val_loss: 0.5773 - val_acc: 0.7322
Epoch 4/20
4086/4086 [==============================] - 7s 2ms/step - loss: 0.5315 - acc: 0.7659 - val_loss: 0.4968 - val_acc: 0.7831
Epoch 5/20
4086/4086 [==============================] - 8s 2ms/step - loss: 0.4588 - acc: 0.7866 - val_loss: 0.4536 - val_acc: 0.7880
Epoch 6/20
4086/4086 [==============================] - 7s 2ms/step - loss: 0.4244 - acc: 0.8005 - val_loss: 0.4611 - val_acc: 0.7913
Epoch 7/20
4086/4086 [==============================] - 7s 2ms/step - loss: 0.4117 - acc: 0.8049 - val_loss: 0.4452 - val_acc: 0.7883
Epoch 8/20
40

In [0]:
encoder_model = Model(encoder_input_placeholder, encoder_outputs)

In [0]:
encoder_outputs_as_input = Input(shape=(maxlen_questions, LATENT_DIM * 2,))
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

In [0]:
context = one_step_attention(encoder_outputs_as_input, initial_s)

In [0]:
decoder_lstm_input = context_last_word_concat_layer([context, decoder_inputs_single_x])


In [0]:
o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[initial_s, initial_c])
decoder_outputs = decoder_dense(o)

In [0]:
decoder_model = Model(
  inputs=[
    decoder_inputs_single,
    encoder_outputs_as_input,
    initial_s, 
    initial_c
  ],
  outputs=[decoder_outputs, s, c]
)


In [0]:
idx2words={v:k for k,v in word2idx_input.items()}

In [0]:
def decode_sequence(input_seq):
  # Encode the input as state vectors.
  enc_out = encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1))
  
  # Populate the first character of target sequence with the start character.
  # NOTE: tokenizer lower-cases all words
  target_seq[0, 0] = word2idx_input['start']

  # if we get this we break
  eos = word2idx_input['end']


  # [s, c] will be updated in each loop iteration
  s = np.zeros((1, LATENT_DIM_DECODER))
  c = np.zeros((1, LATENT_DIM_DECODER))


  # Create the translation
  output_sentence = []
  for _ in range(maxlen_answers):
    o, s, c = decoder_model.predict([target_seq, enc_out, s, c])
        

    # Get next word
    idx = np.argmax(o.flatten())

    # End sentence of EOS
    if eos == idx:
      break

    word = ''
    if idx > 0:
      word = idx2words[idx]
      output_sentence.append(word)

    # Update the decoder input
    # which is just the word just generated
    target_seq[0, 0] = idx

  return ' '.join(output_sentence)



In [166]:
while True:
  # Do some test translations
  i = np.random.choice(len(texts))
  input_seq = padded_questions[i:i+1]
  translation = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', texts[i])
  print('Predicted translation:', translation)
  print('Actual translation:', labels[i])

  ans = input("Continue? [Y/n]")
  if ans and ans.lower().startswith('n'):
    break

-
Input sentence: have you heard kook crazy things original mix by on sound cloud np
Predicted translation: angry
Actual translation: <START> angry <END>
Continue? [Y/n]y
-
Input sentence: i m horrible at studying hate studying give me 3 h o u r s im good
Predicted translation: happy
Actual translation: <START> happy <END>
Continue? [Y/n]y
-
Input sentence: does cbd reduce anxiousness anxiety does cbd reduce anxiousness brought on by consuming high amounts of
Predicted translation: sad
Actual translation: <START> sad <END>
Continue? [Y/n]y
-
Input sentence: teens anxiety how to help your kids anxiety world renowned psychologist has stunning
Predicted translation: sad
Actual translation: <START> sad <END>
Continue? [Y/n]y
-
Input sentence: did you survive the great quake of 2018 worried
Predicted translation: sad
Actual translation: <START> sad <END>
Continue? [Y/n]y
-
Input sentence: hey kindly help me spread my new single to your amazing fans i believe they will love it happy thankyou

KeyboardInterrupt: ignored